In [ ]:
import pandas as pd
import numpy as np

seed = 42

In [ ]:
!pip install pyscipopt
#!pip install imodels

In [ ]:
from collections import defaultdict

from pyscipopt import Branchrule, SCIP_RESULT, Model
import pandas as pd


In [ ]:
from pyscipopt import Model, quicksum
from datetime import datetime



## Dataset loading (read generated pkl files)

In [ ]:
BP_train = pd.read_pickle("dataset/BP_train_solution.pkl")
BP_test = pd.read_pickle("dataset/BP_test_solution.pkl")
SC_train = pd.read_pickle("dataset/SC_train_solution.pkl")
SC_test = pd.read_pickle("dataset/SC_test_solution.pkl")

BP_train = BP_train.sample(frac=1, random_state=seed)
BP_test = BP_test.sample(frac=1, random_state=seed)
SC_train = SC_train.sample(frac=1, random_state=seed)
SC_test = SC_test.sample(frac=1, random_state=seed)

train = pd.concat([BP_train, SC_train], ignore_index=True)
test = pd.concat([BP_test, SC_test], ignore_index=True)

X_train = train.drop(columns=['score'])
Y_train = train['score']

X_test = test.drop(columns=['score'])
Y_test = test['score']

X = pd.concat([X_train, X_test], ignore_index=True)
Y = pd.concat([Y_train, Y_test], ignore_index=True)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

kb = SelectKBest(f_regression, k=10).fit(X, Y)
X_train_k_best = kb.transform(X_train)
k_best_feat = kb.get_feature_names_out()
X_test_k_best = kb.transform(X_test)

## Preprocessing

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train_k_best)
X_train_scaled = scaler.transform(X_train_k_best)
X_test_scaled = scaler.transform(X_test_k_best)

### PCA

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# First PCA visualization
pca1 = PCA(n_components=2)
X_train_pca1 = pca1.fit_transform(X_train_scaled)

plt.figure(figsize=(10, 8))
plt.title('PCA visualization of training data (2 components)')
plt.scatter(X_train_pca1[:, 0], X_train_pca1[:, 1], alpha=0.5)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.show()

## Extra Trees Regressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, max_error

params = {
    "n_estimators": [10, 15, 20],
    "min_samples_leaf": [5, 7, 10],
    "max_depth": [5, 10, 20],
}

et = ExtraTreesRegressor(random_state=seed, n_jobs=4)
gs = GridSearchCV(et, param_grid=params, cv=5, scoring='r2', n_jobs=4)
gs.fit(X_train_scaled, Y_train)

print("Best score: ", gs.best_score_)
print("Best params: ", gs.best_params_)

scores = gs.score(X_test_scaled, Y_test)
print(f"Test set r2 score: {scores:.4f}")
print(f"Test set MSE: {mean_squared_error(Y_test, gs.predict(X_test_scaled)):.4f}")
print(f"Test set MAPE: {mean_absolute_percentage_error(Y_test, gs.predict(X_test_scaled)):.4f}")
print(f"Test set Max error: {max_error(Y_test, gs.predict(X_test_scaled)):.4f}")

## Single decision tree regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, max_error

params = {
    "min_samples_leaf": [5, 7, 10],
    "max_depth": [5, 7, 10],
}

dt = DecisionTreeRegressor(random_state=seed)
gs = GridSearchCV(dt, param_grid=params, cv=5, scoring='r2', n_jobs=4)
gs.fit(X_train_scaled, Y_train)

print("Best score: ", gs.best_score_)
print("Best params: ", gs.best_params_)

scores = gs.score(X_test_scaled, Y_test)
print(f"Test set r2 score: {scores:.4f}")
print(f"Test set MSE: {mean_squared_error(Y_test, gs.predict(X_test_scaled)):.4f}")
print(f"Test set MAPE: {mean_absolute_percentage_error(Y_test, gs.predict(X_test_scaled)):.4f}")
print(f"Test set Max error: {max_error(Y_test, gs.predict(X_test_scaled)):.4f}")

# imodels rules

In [ ]:
!pip install imodels

In [ ]:
from imodels import HSTreeRegressorCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, max_error
from datetime import datetime

mdl = HSTreeRegressorCV(max_leaf_nodes=5)
mdl.fit(X_train_scaled, Y_train, feature_names=k_best_feat)

start = datetime.now()
preds = mdl.predict(X_test_scaled)
end = datetime.now()
print("Time: ", end - start)
print("R2: ", r2_score(Y_test, preds))
print("MSE: ", mean_squared_error(Y_test, preds))
print("MAPE: ", mean_absolute_percentage_error(Y_test, preds))
print("Max error: ", max_error(Y_test, preds))
print(mdl)

In [ ]:
from imodels import GreedyTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, max_error
from datetime import datetime

mdl = GreedyTreeRegressor(min_samples_leaf=5, max_depth=4)
mdl.fit(X_train_scaled, Y_train, feature_names=k_best_feat)

start = datetime.now()
preds = mdl.predict(X_test_scaled)
end = datetime.now()
print("Time: ", end - start)
print("R2: ", r2_score(Y_test, preds))
print("MSE: ", mean_squared_error(Y_test, preds))
print("MAPE: ", mean_absolute_percentage_error(Y_test, preds))
print("Max error: ", max_error(Y_test, preds))
print(mdl)

## RIPPER

In [ ]:
!pip install wittgenstein

In [ ]:
import wittgenstein as lw

ripper_clf = lw.RIPPER()

from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
from datetime import datetime

train_indices = np.random.permutation(X_train_k_best.shape[0])
test_indices = np.random.permutation(X_test_k_best.shape[0])

train_indices = train_indices[:5000]
test_indices = test_indices[:1000]

X_sample_train = X_train_k_best[train_indices]
Y_sample_train = Y_train[train_indices]
X_sample_test = X_test_k_best[test_indices]
Y_sample_test = Y_test[test_indices]

kb = KBinsDiscretizer(n_bins=2, strategy="quantile", encode='ordinal')
Y_train_encoded = kb.fit_transform(Y_sample_train.values.reshape(-1, 1))
Y_test_encoded = kb.fit_transform(Y_sample_test.values.reshape(-1, 1))

ripper_clf.fit(X_sample_train, Y_train_encoded, pos_class=1)  # TODO: add feature names

start = datetime.now()
preds = ripper_clf.predict(X_sample_test)
end = datetime.now()
print("Time: ", end - start)
print("Accuracy: ", accuracy_score(Y_test_encoded, preds))
print("F1 score: ", f1_score(Y_test_encoded, preds))
print("Recall score:", recall_score(Y_test_encoded, preds))
print("Precision score:", precision_score(Y_test_encoded, preds))
print("Confusion matrix", confusion_matrix(Y_test_encoded, preds))
print(ripper_clf.out_model())

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(Y_test_encoded, preds)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2,
         label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve with Cross Validation')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()